In [1]:
import import_ipynb
from arrival_networkx import Arrival, get_branch_instance
import numpy as np
import random as rm
from typing import List
from collections import deque
import networkx as nx

importing Jupyter notebook from arrival_networkx.ipynb


In [2]:
def multi_run_procedure(instance: Arrival, S_subset: List[int]):
    s_curr =  instance.s_0
    s_next = instance.s_1
    
    w = {s:1 for s in S_subset}
    
    t = {s:0 for s in instance.vertices}
    t[0] = 1
    for v in S_subset:
        t[instance.s_0[v]] += np.ceil(w[v]/2)
        t[instance.s_1[v]] += np.floor(w[v]/2)
    # print(t)
    
    waiting_set = []
    for v in instance.vertices:
        if v not in S_subset and v not in [instance.target_node, instance.sink_node]:
            waiting_set.append(v)
            
    while len(waiting_set)>0:
        waiting_set_ = [ws for ws in waiting_set if t[ws]>0]
        if not waiting_set_:
            break
        
        choose = rm.choice(waiting_set_)
        # print(choose,t)
        tau = rm.randint(1,t[choose])
        t[choose] -= tau
        t[s_curr[choose]] += np.ceil(tau/2)
        t[s_next[choose]] += np.floor(tau/2)
        
        if tau & 1 : 
            temp = s_curr[choose]
            s_curr[choose] = s_next[choose]
            s_next[choose] = temp
            
        # break
    print("final t array ",t) 
    return t


In [4]:
def decompose_into_layers(instance : Arrival):
    """
    Decompose the graph into layers based on the distance of the vertices to the destination nodes.
    
    Parameters:
    - graph: A NetworkX graph instance.
    - target_node: The target node (d).
    - sink_node: The sink node (d').
    
    Returns:
    - layers: A dictionary where key is the layer index and value is a list of nodes in that layer.
    - max_dist: The maximum distance (layer index) found.
    """
    
    # Initialize the layers dictionary
    layers = {}
    # Distance dictionary with initial values set to None for each node
    dist = {node: float('inf') for node in instance.graph.nodes()}
    
    # Define a BFS procedure to calculate distances to {target_node, sink_node}
    queue = deque([(instance.target_node, 0), (instance.sink_node, 0)])
    while queue:
        current_node, current_dist = queue.popleft()
        if dist[current_node] == float('inf'):
            dist[current_node] = current_dist
            if current_dist not in layers:
                layers[current_dist] = [current_node]
            else:
                layers[current_dist].append(current_node)
            for neighbor in instance.graph.predecessors(current_node):
                if dist[neighbor] == float('inf'):
                    queue.append((neighbor, current_dist + 1))
    
    # Calculate max_dist
    max_dist = max(dist.values())
    
    return layers, max_dist

In [5]:
def compute_phi_set(instance: Arrival, phi: float) :
    layers, max_dist = decompose_into_layers(instance) 
    print(layers)
    S = []
    U = layers[0]
    for i in range(1,len(layers)):
        if len(layers[i]) < phi*len(U):
            S += layers[i]
            U = []
        U += layers[i]
    return S
    

In [6]:
def subexponential(instance: Arrival, phi: float):
    S = compute_phi_set(instance, phi)
    print("S is ",S)
    t = multi_run_procedure(instance, S)
    return t

In [16]:
hard_instance = get_branch_instance(100,0.5)
subexponential(hard_instance, 0.5)